# MODS 203 : Pre-Processing

This pre-processing focuses on the sale of coffee machines on [Amazon](https://www.amazon.fr/s?k=coffee+machine&crid=1CZR37USHHBL&sprefix=coffee%2Caps%2C152&ref=nb_sb_ss_ts-doa-p_2_6). The data was scrapped and then compiled into an xlsx file.

## Table of Contents
* [1. Importing libraries](#chapter1)
    $\\$
* [2. Downloading the DataSet](#chapter2)
    $\\$
* [3. DataSet Overview](#chapter3)
    $\\$
* [4. Pre-Processing](#chapter4)
    * [4.1 Removing duplicates](#section_4_1)
    * [4.2 Adapting Time formats](#section_4_2) 
        * [4.2.1 Feature - `delivery`](#sub_section_4_2_1)
        * [4.2.2 Feature - `Date First Available`](#sub_section_4_2_2)
    * [4.3 Dealing with `Capacity`](#section_4_3)
    * [4.4 OHE on the `Coffee Maker Type`](#section_4_4)
    * [4.5 Using booleans regarding the `Filter Type`](#section_4_5)
    * [4.6 OHE on the `Material`](#section_4_6)
    * [4.7 Programmable ? making use of the `Special Feature` column](#section_4_7)
    * [4.8 Retrieving informations on the `Brand` feature (WIP)](#section_4_8)
    * [4.9 Quick cleaning - `Wattage` / `Voltage` / `Item Weight`](#section_4_9)
        * [4.9.1 Feature - `Wattage`](#sub_section_4_2_1)
        * [4.9.2 Feature - `Voltage`](#sub_section_4_2_2)
        * [4.9.3 Feature - `Item Weight`](#sub_section_4_2_2)
    * [4.10 OHE on the `Human Interface Input`](#section_4_10)
    * [4.11 Cleaning the `Colors`](#section_4_11)
    $\\$
* [5. Saving the final DataFrame](#chapter5)

_Note : WIP - Work In Progress_


<a class="anchor" id="chapter1"></a>
## 1. Importing libraries 

In [2]:
from tabulate import tabulate
from string import digits
import pandas as pd
import webcolors
import math
import re

If you have not already donwloaded [webcolors](https://pypi.org/project/webcolors/), please run the following cell :

In [3]:
# %pip install webcolors

<a class="anchor" id="chapter2"></a>
## 2. Downloading the DataSet

In [4]:
# Downloading the un-processed DataSet :
file_xlsx = 'out7.xlsx'

In [5]:
# First look at the DataSet :
df = pd.read_excel(file_xlsx)
df.head(2)

,price,rating,timesRated,title,link,page,position,delivery,other__Brand,other__Capacity,...,other__Domestic Shipping,other__Country of Origin,other__Item model number,other__Batteries,other__Material Care Instructions,other__Assembly Required,other__Number of Pieces,other__Batteries Required?,other__Is Discontinued By Manufacturer,other__Date First Available
0,849.00,4,4948,PHILIPS 3200 Series Fully Automatic Espresso M...,https://www.amazon.com/gp/slredirect/picassoRe...,1,1,"Monday, January 9",PHILIPS,3.8 Pounds,...,NaN,Romania,EP3241/74,NaN,NaN,NaN,NaN,NaN,NaN,"July 26, 2022"
1,149.99,1,1728,"10-Cup Drip Coffee Maker, Grind and Brew Autom...",https://www.amazon.com/gp/slredirect/picassoRe...,1,3,January 18 - 26,Gevi,10 Cups,...,NaN,NaN,9406,NaN,NaN,NaN,NaN,NaN,NaN,"April 21, 2017"


<a class="anchor" id="chapter3"></a>
## 3. DataSet Overview

In [6]:
# Shape of the original DataSet :
print(f'Shape of the original DataSet : {df.shape}')
print('\n')

# Columns inventary : 
print('======================= Columns Inventary =======================')
print(df.columns)

Shape of the original DataSet : (612, 48)


======================= Columns Inventary =======================
Index(['price', 'rating', 'timesRated', 'title', 'link', 'page', 'position',
       'delivery', 'other__Brand', 'other__Capacity', 'other__Color',
       'other__Item Dimensions LxWxH', 'other__Special Feature',
       'other__Coffee Maker Type', 'other__Product Dimensions',
       'other__Material', 'other__Filter Type',
       'other__Specific Uses For Product',
       'other__Recommended Uses For Product', 'other__Style',
       'other__Number of settings', 'other__Number of Programs',
       'other__Included Components', 'other__Operation Mode', 'other__Wattage',
       'other__Voltage', 'other__Model Name', 'other__Number of Items',
       'other__Unit Count', 'other__Department',
       'other__Human Interface Input', 'other__Package Type',
       'other__Package Dimensions', 'other__Item Weight',
       'other__Power Source', 'other__Exterior Finish', 'other__Manufacture

Some features contain the substring `'other__'`. For greater visibility, we will remove this subset of characters.

In [7]:
# Renaming columns :
df.columns = [col.replace('other__','') for col in df.columns]
print('======================= New Columns Inventary =======================')
print(df.columns)

======================= New Columns Inventary =======================
Index(['price', 'rating', 'timesRated', 'title', 'link', 'page', 'position',
       'delivery', 'Brand', 'Capacity', 'Color', 'Item Dimensions LxWxH',
       'Special Feature', 'Coffee Maker Type', 'Product Dimensions',
       'Material', 'Filter Type', 'Specific Uses For Product',
       'Recommended Uses For Product', 'Style', 'Number of settings',
       'Number of Programs', 'Included Components', 'Operation Mode',
       'Wattage', 'Voltage', 'Model Name', 'Number of Items', 'Unit Count',
       'Department', 'Human Interface Input', 'Package Type',
       'Package Dimensions', 'Item Weight', 'Power Source', 'Exterior Finish',
       'Manufacturer', 'ASIN', 'Domestic Shipping', 'Country of Origin',
       'Item model number', 'Batteries', 'Material Care Instructions',
       'Assembly Required', 'Number of Pieces', 'Batteries Required?',
       'Is Discontinued By Manufacturer', 'Date First Available'],
      dt

### **Overview of the features :**
|Name |Type |Description|
|--|--|--|
|price|float|Price of the product in XXX.|
|rating|int|Customer's rating out of 5.|
|timesRated|int|Number of times the object was rated.|
|title|string|Title of the product.|
|link|string|Link to the product page.|
|page|int|Page position.|
|position|int|Article position regarding its visibility.|
|delivery|string|Delivery date if purchased right away.|
|Brand|string|Brand of the article.|
|Capacity|string|Capacity of the product.|
|Color|string|Color of the product.|
|Item Dimensions LxWxH|string|Dimensions of the item.|
|Special Feature|string|Description of potential special features.|
|Coffee Maker Type|string|Type of the coffee machine (Expresso, Drip, Moka,...).|
|Product Dimensions|string|Dimensions of the product in inches.|
|Material|string|Material used for the product.|
|Filter Type|string|Type of filter to use for the cofee machine.|
|Specific Uses For Product|string|Tags for the product.|
|Recommended Uses For Product|string|Advice for the customer when using the product.|
|Style|string|description of the style of the coffee machine.|
|Number of settings|int|Number of settings for the product.|
|Number of Programs|int|Number of programs for the product.|
|Included Components|string|Additional components sold with the machine.|
|Operation Mode|string|Operation mode of the coffee machine.|
|Wattage|string|Wattage of the product.|
|Voltage|string|Required voltage for the product.|
|Model Name|string|Name of the machine model.|
|Number of Items|int|Number of items sold per package.|
|Unit Count|string|Unit Count XXXX ?|
|Department|string|Tag for the coffee machine.|
|Human Interface Input|string|type of interface provided when interacting with the machine.|
|Package Type|string|Type of packaging.|
|Package Dimensions|string|Package dimensions.|
|Item Weight|string|Weight of the product in pounds.|
|Power Source|string|Power source of the product.|
|Exterior Finish|string|Exterior finish of the product.|
|Manufacturer|string|Manufacturer of the product.|
|ASIN|string|Amazon Standard Identification Number.|
|Domestic Shipping|string|Details regarding where the item can be shipped.|
|Country of Origin|string|Country of origin of the product.|
|Item model number|string|Model number of the product.|
|Batteries|string|Model of the batteries (if required).|
|Material Care Instructions|string|Instructions on how to maintain the equipment.|
|Assembly Required|boolean|Assembly required?|
|Number of Pieces|int|Number of Pieces.|
|Batteries Required?|boolean|Batteries Required?|
|Is Discontinued By Manufacturer|boolean|Is Discontinued By Manufacturer.|
|Date First Available|string|Date when the announce was first released on Amazon.|

In [8]:
# Statistical report on numerical features :
df.describe()

,price,page,position,Number of settings,Number of Programs,Number of Items,Number of Pieces
count,612.000000,612.000000,612.000000,2.0,3.0,391.000000,3.0
mean,177.855686,23.400327,29.434641,6.0,4.0,1.104859,1.0
std,262.276629,116.970536,17.809245,0.0,0.0,1.065425,0.0
min,24.510000,1.000000,1.000000,6.0,4.0,1.000000,1.0
25%,49.990000,2.000000,13.000000,6.0,4.0,1.000000,1.0
50%,95.200000,3.000000,29.000000,6.0,4.0,1.000000,1.0
75%,170.487500,4.000000,45.000000,6.0,4.0,1.000000,1.0
max,2995.000000,697.000000,60.000000,6.0,4.0,20.000000,1.0


In [9]:
# Checking the number of NaN values per category :
print('======= NaN values per category =======')
df.isna().sum()[df.isna().sum() > 0]

======= NaN values per category =======


timesRated                         342
Brand                               63
Capacity                            95
Color                               93
Item Dimensions LxWxH              608
Special Feature                    107
Coffee Maker Type                  168
Product Dimensions                  63
Material                           116
Filter Type                        134
Specific Uses For Product          236
Recommended Uses For Product       557
Style                              275
Number of settings                 610
Number of Programs                 609
Included Components                238
Operation Mode                     561
Wattage                            174
Voltage                            195
Model Name                         185
Number of Items                    221
Unit Count                         479
Department                         591
Human Interface Input              198
Package Type                       320
Package Dimensions       

<a class="anchor" id="chapter4"></a>
## 4. Pre-Processing

<a class="anchor" id="section_4_1"></a>
### 4.1 Removing duplicates :

We first remove any possible duplicates.

In [10]:
df.drop_duplicates(inplace=True)

<a class="anchor" id="section_4_2"></a>
### 4.2 Adapting Time formats :

Some of the characteristics refer to dates, we will transfer them using the [datetime](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html) format. The columns concerned are :
* `delivery`
* `Date First Available`

<a class="anchor" id="sub_section_4_2_1"></a>
#### 4.2.1 Feature - `delivery`

In [11]:
# Modification for dates using a range
def adapt_DateFormat(date):
    if (date.find('-')!=-1):
        return date[:date.find('-')-1]
    else : return date

In [12]:
# We divide our DataSet into two subsets depending on the date format used
format1 = pd.to_datetime(df['delivery'], format='%A, %B %d', errors='coerce')
format2 = pd.to_datetime(df['delivery'], format='%B %d', errors='coerce')

# We then merge the two subsets into our original DataSet
df['delivery'] = format1.fillna(format2)
df = df[df['delivery'].notnull()]

df['delivery'] = df['delivery'].apply(lambda date : date.replace(year=2023))
df['delivery'].head(3)


0   2023-01-09
2   2023-01-09
3   2023-01-13
Name: delivery, dtype: datetime64[ns]

<a class="anchor" id="sub_section_4_2_2"></a>
#### 4.2.2 Feature - `Date First Available`

We apply the same processus to the `Date First Available` feature.

In [13]:
df = df[df['Date First Available'].notnull()]

DFA_mask = df['Date First Available'].apply(lambda date : date.replace(' ',''))
df['Date First Available'] = pd.to_datetime(DFA_mask, format='%B%d,%Y')

df['Date First Available'].head(3)

0   2022-07-26
2   2021-09-14
3   2019-12-15
Name: Date First Available, dtype: datetime64[ns]

<a class="anchor" id="section_4_3"></a>
### 4.3 Dealing with `Capacity` :

The `Capacity` feature needs to be harmonised. Indeed, capacity is expressed in different units (pounds, litres, cups, mililitres, ounces, ...). We will express all capacities in litres. 

In [14]:
# Examples of units :
df['Capacity'].head(3)

0          3.8 Pounds 
2     96 Fluid Ounces 
3             12 Cups 
Name: Capacity, dtype: object

We create a dictionnary containing the conversion values.

In [15]:
# Dictionnary containing the conversion values :
dict_conv_c = {
    'pounds' : 0.45,
    'cups' : 0.236588,
    'fluidounces': 0.0295735,
    'milliliters': 0.001,
    'quarts': 0.946353,
    'cubiccentimeters' : 0.001,
    'kilograms' : 1,
    'cubicinches' : 0.0163871,
    'liters' : 1,
    'ounces' : 0.0295735,
    'cubicfeet' : 28.3168,
    'gallons' : 3.78541,
}

Each string contains different types of information:
* a float characterising the capacity: `cap_f`. 
* a string indicating the unit used to express the capacity: `cap_u`.

We automate the process of extracting this data.

The technique used below when using `.translate(remove_digits)` works on python 3. For python 2 please refer to the following [comments](https://stackoverflow.com/questions/12851791/removing-numbers-from-string).

In [16]:
remove_digits = str.maketrans('', '', digits)

# Extract key informations including value and unit :
def conversion_extractor(cap):
    if not(pd.isnull(cap)) and (cap.find(',')==-1):
        cap_lk = cap.lower()
        # capacity float cap_f
        cap_f = float(re.findall(r'\d+(?:\.\d+)?', cap_lk)[0])
        # capacity unit cap_u
        cap_u = cap_lk.replace(str(cap_f), '')
        cap_u = cap_u.replace(' ','')
        cap_u = cap_u.translate(remove_digits)
        # results
        return cap_f, cap_u
    else: return 'None', 'None'

In [17]:
# NaN values will appear as -1 in the final column :
def convert_to_liters(cap_f, cap_u):
    if (cap_f == 'None'):
        return -1
    else: 
        capacity_final = cap_f*dict_conv_c.get(cap_u)
        return capacity_final

We can now create our final conversion function and test it on possible error cases.

In [18]:
# Final function :
def auto_convertor(cap):
    cap_f, cap_u = conversion_extractor(cap)
    return convert_to_liters(cap_f, cap_u)

test1, test2, test3 = '3.8 Pounds', None, '14 Ounces, 8 Ounces'
test_converted1, test_converted2, test_converted3 = auto_convertor(test1), auto_convertor(test2), auto_convertor(test3)

boolean_test1 = (test_converted1 == 1.71)
boolean_test2 = (test_converted2 == -1)
boolean_test3 = (test_converted3 == -1)

# Results : 
results = []
results.append(['Test 1', 'General case', boolean_test1])
results.append(['Test 2', 'None value case', boolean_test2])
results.append(['Test 3', 'Multiple values case', boolean_test3])

# Test Report : 
print('================= Test Report ================')
print(tabulate(results, headers=[' ', 'type of case', 'test succeeded']))

================= Test Report ================
        type of case          test succeeded
------  --------------------  ----------------
Test 1  General case          True
Test 2  None value case       True
Test 3  Multiple values case  True


Eventually we apply the entire process to our DataFrame.

In [19]:
# New feature column : Capacity (in liters) 
df['Capacity (in liters)'] = df['Capacity'].apply(lambda cap : auto_convertor(cap))

# Checking the first 4 values :
df['Capacity (in liters)'].head(4)

0    1.710000
2    2.839056
3    2.839056
4    1.125000
Name: Capacity (in liters), dtype: float64

<a class="anchor" id="section_4_4"></a>
### 4.4 OHE on the `Coffee Maker Type` :

In [20]:
df['Coffee Maker Type'].unique()

array([' Espresso Machine ', ' Drip Coffee Machine ', ' Pour Over ',
       ' Moka Pot ', ' Vacuum Coffee Pot ', nan,
       ' Turkish Coffee Machine ', ' Coffee Infuser ', ' Percolator ',
       ' French Press '], dtype=object)

In [21]:
# The studied types have unuseful spaces at the beginning and end of each string 
def remove_spaces(type):
    if not(pd.isnull(type)):
        return type[1:-1]
    else : return

Let's take a look at the different types of coffee machines sold on Amazon.

In [22]:
# Inventary of all possible types :
df['Coffee Maker Type'] = df['Coffee Maker Type'].apply(lambda type : remove_spaces(type))
df['Coffee Maker Type'].unique()

array(['Espresso Machine', 'Drip Coffee Machine', 'Pour Over', 'Moka Pot',
       'Vacuum Coffee Pot', None, 'Turkish Coffee Machine',
       'Coffee Infuser', 'Percolator', 'French Press'], dtype=object)

We can now use a hot coding method to make it possible to use the `Coffee maker type` function for future analysis.

In [23]:
pd_data_dum = pd.get_dummies(df, prefix='', prefix_sep='', columns=['Coffee Maker Type'])
pd_data_dum.head(3)

,price,rating,timesRated,title,link,page,position,delivery,Brand,Capacity,...,Capacity (in liters),Coffee Infuser,Drip Coffee Machine,Espresso Machine,French Press,Moka Pot,Percolator,Pour Over,Turkish Coffee Machine,Vacuum Coffee Pot
0,849.00,4,4948,PHILIPS 3200 Series Fully Automatic Espresso M...,https://www.amazon.com/gp/slredirect/picassoRe...,1,1,2023-01-09,PHILIPS,3.8 Pounds,...,1.710000,0,0,1,0,0,0,0,0,0
2,213.99,5,5848,Ninja CFP301 DualBrew Pro System 12-Cup Coffee...,https://www.amazon.com/Ninja-CFP301-Coffee-Mak...,1,4,2023-01-09,Ninja,96 Fluid Ounces,...,2.839056,0,1,0,0,0,0,0,0,0
3,34.75,1,1874,"BLACK+DECKER 12-Cup* Programmable Coffeemaker,...",https://www.amazon.com/BLACK-DECKER-Programmab...,1,15,2023-01-13,BLACK+DECKER,12 Cups,...,2.839056,0,0,1,0,0,0,0,0,0


<a class="anchor" id="section_4_5"></a>
### 4.5 Using booleans regarding the `Filter Type` :

Basically, the `Filter Type` informs us on wether reusable capsules are used or it is a paper type filter. We arbitrarly choose the following convention :
* $1$  : Reusable
* $0$  : Paper
* $-1$ : No information

In [24]:
# Function used to encode our new Filter Type feature
def filter_type_convertor(filt_type):
    if (pd.isnull(filt_type)):
        return -1
    elif (filt_type.find('Reusable')) or (filt_type.find('reusable')):
        return 1
    elif (filt_type.find('Paper')) or (filt_type.find('paper')):
        return 0
    else : return -1

We now apply the previous function.

In [25]:
df['Filter Type'] = df['Filter Type'].apply(lambda filt_type : filter_type_convertor(filt_type))

<a class="anchor" id="section_4_6"></a>
### 4.6 Cleaning `Material` :

We will first clean up the DataSet to get a clearer view of the materials used in each coffee machine.
In each string, we will try to identify key words and simplify the actual rows.

In [26]:
# First look at all the materials used
df['Material'].unique()

array([nan, ' Plastic ', ' Stainless Steel ', ' Ceramic ',
       ' Nylon (PA66), Polypropylene, Silicone, Acrylonitrile Butadiene Styrene, Polycarbonate ',
       ' Glass ', ' Stainless Steel, Plastic ',
       ' Polypropylene, Stainless Steel, Glass ', ' Metal ',
       ' Plastic, Metal, Rubber ', ' Glass, Plastic ', ' Stoneware ',
       ' Steel ', ' Stainless Steel, Glass ', ' Polypropylene ',
       ' Stainless Steel, Aluminum, Plastic, Metal ',
       ' Aluminum, Stainless Steel ', ' Stainless Steel, Glass, Plastic ',
       ' Stainless Steel, Copper ',
       ' Stainless Steel+plastic+Aluminum(Espresso Outlet Part) ',
       ' Chrome, Plastic ', ' MU-CM10 ', ' Plastic, Metal, Copper ',
       ' Aluminum ', ' Acrylonitrile Butadiene Styrene ',
       ' Plastic, Metal ', ' Stainless Steel, Plastic, Metal ', ' 塑料 ',
       ' Stainless Steel, Borosilicate Glass ',
       ' espresso machine with milk frother, Stainless Steel, small espresso machine, cafeteras electricas expresso, Exp

In [27]:
materials_plastic = [
    'plastic',
    '塑料', # Plastic in chinese
    'abs', # styrenic polymers - Plastic 
    'polypropylene', # thermoplastic polymer - Plastic 
]

materials_metal = [
    'metal', 
    'aluminum',
    'copper',
]

materials_glass = [
    'borosilicate',
    'glass',
]

We could imagine five new featurs :
* `contains plastic`
* `contains metal`
* `contains rubber`
* `contains ceramic`
* `contains glass`

In [28]:
def contains_material(material, list_mat):
    if (pd.isnull(material)):
        return 0
    else:
        material_lk = material.lower()
        for mat in list_mat:
            if (material_lk.find(mat)!=-1): 
                return 1
        return 0

Creating such features. We consider booleans columns :
* $1$ if the product contains a specific material.
* $0$ if not.

In [29]:
# We can now create new features based on the materials found in the products :
df['contains_plastic'] = df['Material'].apply(lambda material : contains_material(material, materials_plastic))
df['contains_metal']   = df['Material'].apply(lambda material : contains_material(material, materials_metal))
df['contains_glass']   = df['Material'].apply(lambda material : contains_material(material, materials_glass))
df['contains_ceramic'] = df['Material'].apply(lambda material : contains_material(material, ['ceramic']))
df['contains_rubber']  = df['Material'].apply(lambda material : contains_material(material, ['rubber']))


<a class="anchor" id="section_4_7"></a>
### 4.7 Programmable ? making use of the `Special Feature` column :

One particular feature seems to be recurrent and could be a key factor in understanding the popularity of a product: is the coffee machine programmable?

The following code structure is quite similar to the one we have already done for materials.

In [30]:
def is_programmable(spe_feature):
    if (pd.isnull(spe_feature)):
        return 0
    else:
        spe_feature_lk = spe_feature.lower()
        if spe_feature_lk.find('programmble'):
            return 1
        return 0

We can now implement a new feature : `is_programmable`

In [31]:
df['is_programmable'] = df['Special Feature'].apply(lambda spe_feature : is_programmable(spe_feature))
print(f'* Number of programmable coffee machines in the DataSet : {df["is_programmable"].sum()}')

* Number of programmable coffee machines in the DataSet : 463


<a class="anchor" id="section_4_8"></a>
### 4.8 Retrieving informations on the `Brand` feature (WIP):

**Possible important features :**
* Country / Region of the brand.
* Find a way to say if the brand is a "big" brand or a "small" company. 

In [32]:
df['Brand'].unique()

array([' PHILIPS ', ' Ninja ', ' BLACK+DECKER ', ' Bonavita ', ' CYETUS ',
       ' Chefman ', ' Mr. Coffee ', ' Philips Kitchen Appliances ',
       ' Famiworths ', ' OXO ', ' Gevi ', ' Dolce Gusto ',
       ' Elite Gourmet ', ' Breville ', " De'Longhi ", ' SOWTECH ',
       ' AdirChef ', nan, ' Keurig ', ' Taylor Swoden ', ' Zojirushi ',
       ' Instant Pot ', ' Carivia ', ' Bonsenkitchen ', ' Amazon Basics ',
       ' Nespresso-Breville ', ' BUNN ', ' Nestle Nespresso ', ' KRUPS ',
       ' Cuisinart ', " L'OR ", ' Zulay Kitchen ', ' Hamilton Beach ',
       ' ICUIRE ', ' Mecity ', ' SHARDOR ', ' Café ', ' COSIKIE ',
       ' Technivorm Moccamaster ', ' SYBO ', ' VIMUKUN ', ' Teglu ',
       ' wirsh ', ' Yabano ', ' Boly ', ' BELLA ', ' Sincreative ',
       ' AIRMSEN ', ' Kwister ', ' Capresso ', ' CucinaPro ',
       ' Mueller Austria ', ' avigator ', ' Smeg ', ' SIFENE ',
       ' KitchenBro ', ' Gaggia ', ' Wamife ', ' Aiosa ', ' Mcilpoog ',
       ' Presto ', ' kognita ', ' Mi

_Amelioration : Using an API to retrieve important data on the following brands ?_

In [33]:
dict_brands = {
        'PHILIPS',
        'Ninja',
        'BLACK+DECKER',
        'Bonavita',
        'CYETUS',
        'Chefman',
        'Mr.Coffee' ,
        'Philips Kitchen Appliances' ,
        'Famiworths' ,
        'OXO' ,
        'Gevi' ,
        'Dolce Gusto' ,
        'Elite Gourmet',
        'Breville' ,
        "DeLongh'i",
        'SOWTECH' ,
        'AdirChef' ,
        "na'n",
        'Keurig' ,
        "Taylor 'Swoden",
        'Zojirushi' ,
        "Instant 'Pot" ,
        'Carivia' ,
        'Bonsenkitchen' ,
        'Amazon Basics' ,
        'Nespresso-Breville',
        'BUNN' ,
        'Nestle Nespresso',
        'KRUPS' ,
        'Cuisinart' ,
        "LO'R",
        'Zulay Kitchen' ,
        'Hamilton Beach',
        'ICUIRE' ,
        'Mecity' ,
        'SHARDOR' ,
        'Café' ,
        'COSIKIE' ,
        'Technivorm Moccamaster',
        'SYBO' ,
        'VIMUKUN' ,
        'Teglu' ,
        'wirsh' ,
        'Yabano' ,
        'Boly' ,
        'BELLA' ,
        'Sincreative' ,
        'AIRMSEN' ,
        'Kwister' ,
        'Capresso' ,
        'CucinaPro' ,
        'Mueller Austria',
        'avigator' ,
        'Smeg' ,
        'SIFENE' ,
        'KitchenBro' ,
        'Gaggia' ,
        'Wamife' ,
        'Aiosa' ,
        'Mcilpoog' ,
        'Presto' ,
        'kognita' ,
        'Mixpresso' ,
        'Vimukun' ,
        'CASABREWS' ,
        'Braun' ,
        'Nostalgia' ,
        'Calphalon' ,
        'Xeternity-Made' ,
        'Proctor Silex' ,
        'WantJoin' ,
        'Hario' ,
        'Lavazza' ,
        'Stamo' ,
        'Intasting' ,
        'Russell Hobbs' ,
        'ETHNIQ' ,
        'Toddy' ,
        'GE' ,
        'Sunvivi' ,
        'SUMSATY' ,
        'Geek Chef' ,
        'X WINDAZE' ,
        'CERA+' ,
        'Sulypo' ,
        'GrowlerWerks' ,
        'JoooDeee' ,
        'memoscan' ,
        'LITIFO' ,
        'Tredy' ,
        'West Bend' ,
        'Brim' ,
        'Beko' ,
        'Arzum Okka' ,
        'Hamiton Beach' ,
        'Idealforce' ,
        'BOSCARE' ,
        'Brentwood' ,
        'CHULUX' ,
        'Hamilton Beach Professional' ,
        'JAVASTARR' ,
        'Coffee Gator' ,
        'FOHERE' ,
        'Jura' ,
        'Frigidaire' ,
        'LEONEBEBE' ,
        'Miele' ,
        'HIBREW' ,
        'Better Chef' ,
        'Bogner' ,
        'HAUSWIRT' ,
        'KINGTOO' ,
        'WACACO' ,
        'bonVIVO' ,
        "Rati'o",
}

<a class="anchor" id="section_4_9"></a>
### 4.9 Quick cleaning - `Wattage` / `Voltage` / `Item Weight` :

This section consists in convertin string features into numerical values. To do-so, we will use a similar approach as the one used for the capacity conversions.

<a class="anchor" id="sub_section_4_9_1"></a>
#### 4.9.1 Feature - `Wattage`

In [34]:
def extract_wattage(watt):
    if not(pd.isnull(watt)):
        watt_lk = watt.lower()
        watt_f = math.floor(float(re.findall(r'\d+(?:\.\d+)?', watt_lk)[0]))
        return watt_f
    else: return

In [35]:
# Adding converted feature
df['Wattage'] = df['Wattage'].apply(lambda watt : extract_wattage(watt))
df['Wattage'].tail(3)

608    1400.0
610       NaN
611       NaN
Name: Wattage, dtype: float64

<a class="anchor" id="sub_section_4_9_2"></a>
#### 4.9.2 Feature - `Voltage`

In [36]:
def extract_voltage(watt):
    if not(pd.isnull(watt)) and (watt.find(',')==-1):
        watt_lk = watt.lower()
        watt_f = math.floor(float(re.findall(r'\d+(?:\.\d+)?', watt_lk)[0]))
        if (watt_f > 1000):
            watt_f = int(str(watt_f)[:2])
        return watt_f
    elif not(pd.isnull(watt)) and not(watt.find(',')==-1):
        watt_lk = watt[0:watt.find(',')-1].lower()
        watt_f = math.floor(float(re.findall(r'\d+(?:\.\d+)?', watt_lk)[0]))
        return watt_f
    else: return

In [37]:
# Adding converted feature
df['Voltage'] = df['Voltage'].apply(lambda watt : extract_voltage(watt))
df['Voltage'].tail(3)

608    120.0
610      NaN
611      NaN
Name: Voltage, dtype: float64

<a class="anchor" id="sub_section_4_9_3"></a>
#### 4.9.2 Feature - `Item Weight`

Same issue that we dealt when cleaning the `capacity` feature : some weights are in pounds, others in ounces. We'll use a same approach. We convert all weights in kg.

In [38]:
# Dictionnary containing the conversion values :
dict_conv_w = {
    'pounds' : 2.20462,
    'ounces' : 0.0295735,
    'kg' : 0.028349,
}

In [39]:
remove_digits = str.maketrans('', '', digits)

# Extract key informations including value and unit :
def conversion_extractor_2(weight):
    if not(pd.isnull(weight)):
        weight_lk = weight.lower()
        # weight float weight_f
        weight_f = float(re.findall(r'\d+(?:\.\d+)?', weight_lk)[0])
        # weight unit weight_u
        weight_u = weight_lk.replace(str(weight_f), '')
        weight_u = weight_u.replace(' ','')
        weight_u = weight_u.translate(remove_digits)
        # results
        return weight_f, weight_u
    else: return 'None', 'None'

In [40]:
# NaN values will appear as -1 in the final column :
def convert_to_kg(weight_f, weight_u):
    if (weight_f == 'None'):
        return -1
    else: 
        weight_final = weight_f*dict_conv_w.get(weight_u)
        return weight_final

In [41]:
# Final function :
def auto_convertor_2(weight):
    weight_f, weight_u = conversion_extractor_2(weight)
    return convert_to_kg(weight_f, weight_u)

In [42]:
# Adding converted feature
df['Item Weight (in kg)'] = df['Item Weight'].apply(lambda weight : auto_convertor_2(weight))
df['Item Weight (in kg)'].head(3)

0    52.029032
2    21.362768
3    12.764750
Name: Item Weight (in kg), dtype: float64

<a class="anchor" id="section_4_10"></a>
### 4.10 OHE on the `Human Interface Input` :

The `Human Interface Input` includes only a few different possibilities, including :
* `Buttons`
* `Touchscreen`
* `Dial`
* `Touch Pad`

The approach we will use is quite similar to the one we have already used in [4.6 Cleaning `Material`](#section_4_6).


In [43]:
def contains_hum_int(hum_int, list_hum_int):
    if (pd.isnull(hum_int)):
        return 0
    else:
        hum_int_lk = hum_int.lower()
        print(hum_int_lk)
        if (hum_int.find('unknown')!=-1):
            return 0
        for interf in list_hum_int:
            if (hum_int_lk.find(interf)!=-1): 
                return 1
        return 0

In [44]:
# Plotting the different values of HII :
df['Human Interface Input'].unique()

array([' Buttons ', ' Touchscreen ', nan, ' Touchscreen, Buttons ',
       ' Dial ', ' Dial, Buttons ', ' Unknown ',
       ' Touchscreen, Touch Pad '], dtype=object)

In [45]:
# We can now create new features based on the materials found in the products :
df['HII_buttons']     = df['Human Interface Input'].apply(lambda hum_int : contains_hum_int(hum_int, ['buttons']))
df['HII_dial']        = df['Human Interface Input'].apply(lambda hum_int : contains_hum_int(hum_int, ['dial']))
df['HII_touchscreen'] = df['Human Interface Input'].apply(lambda hum_int : contains_hum_int(hum_int, ['touchscreen']))
df['HII_touch_pad']   = df['Human Interface Input'].apply(lambda hum_int : contains_hum_int(hum_int, ['touch pad']))

 buttons 
 buttons 
 buttons 
 touchscreen 
 buttons 
 buttons 
 buttons 
 touchscreen 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 touchscreen, buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 touchscreen 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 dial 
 buttons 
 touchscreen 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 touchscreen 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 touchscreen 
 buttons 
 touchscreen 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 touchscreen 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 dial, buttons 
 buttons 
 buttons 
 touchscreen 
 buttons 
 buttons 
 buttons 
 touchscreen, buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 buttons 
 touchscreen 
 buttons 
 buttons 
 touchscree

<a class="anchor" id="section_4_11"></a>
### 4.11 Standardizing the `Product Dimensions` :

In the DataSet, we find different formats including, 
* `8.3 x 11 x 12.6 inches`
*  `9.92"D x 13.07"W x 6"H`  

We first create an extract function to retrieve key informations. We will use the following format :
$$
\text{floatD}\space\space\space\text{x}\space\space\space\text{floatW}\space\space\space\text{x}\space\space\space\text{floatH}
$$

In [46]:
# Testing a possible regular expression :
cap_lk = '9.92"D x 13.07"W x 6"H'
re.findall(r'\d+(?:\.\d+)?', cap_lk)

['9.92', '13.07', '6']

In [47]:
# Extractor function :
def extract_dimensions(dim):
    if not(pd.isnull(dim)):
        dim_lk = dim.lower()
        dim_fl = re.findall(r'\d+(?:\.\d+)?', dim_lk)
        dim_f_tot = dim_fl[0]+' x '+dim_fl[1]+' x '+dim_fl[2]
        return dim_f_tot
    else: return


In [48]:
df['Product Dimensions'] = df['Product Dimensions'].apply(lambda dim : extract_dimensions(dim))
df['Product Dimensions'].head(3)

0    19.29 x 11.42 x 18.9
2    9.13 x 11.39 x 15.54
3         10.3 x 7.8 x 12
Name: Product Dimensions, dtype: object

<a class="anchor" id="section_4_11"></a>
### 4.11 Cleaning the `Colors` :

We try to streamline the visualisation of colours. We use a comma separated format.

In [49]:
# First look at the colours :
df['Color'].unique()

array([' Black ', ' Stainless Steel ', ' dark grey ', ' Silver ',
       ' Brushed Stainless Steel ', ' Magnifica ESAM3300 ', ' Blk ',
       ' Green ', nan, ' Chrome ', ' Matte Black ', ' Grey ',
       ' Ink Black ', ' white/stainless steel ',
       ' Black+Stainless Steel ', ' black ', ' Black/Stainless Steel ',
       ' Black/Stainless ', ' Dark Charcoal ', ' Brushed Slate ',
       ' Black with Black Screen ', ' White ', ' Black, Silver ',
       ' Black,Silver ', ' Copper ', ' Black &amp; Stainless Steel ',
       ' Evening Teal ', ' Polished Silver ', ' Stainless ',
       ' black/stainless steel ', ' Aqua ', ' Burgundy ',
       ' Black Stainless ', ' Coffee Maker ',
       ' stainless steel + black plastic ', ' Black Thermal ',
       ' Matte White ', ' Stainless steel Blk ', ' Orange ', ' Cream ',
       ' Black Latte ', ' Dark Black ',
       ' Glaze Black+Removable reservoir ', ' Blue Coffee Machine ',
       ' Clear ', ' Black+Stainless ', ' Blue ', ' Austere Silver ',
  

In [50]:
# Adding data to the already existing colors dictionnary
webcolors.CSS3_NAMES_TO_HEX['stainless'] = '#CFD4D9'
webcolors.CSS3_NAMES_TO_HEX['copper'] = '#B87333'
webcolors.CSS3_NAMES_TO_HEX['nickel'] = '#727472'
webcolors.CSS3_NAMES_TO_HEX['blk'] = '#000000'
webcolors.CSS3_NAMES_TO_HEX['b'] = '#000000'

# Dictionnary (abbreviation / full Name)
dict_color_rebrand = {
    'stainless': 'stainless steel',
    'blk': 'black',
}

In [51]:
remove_digits = str.maketrans('', '', digits)

def color_rebrand(color_format):
    for elt in dict_color_rebrand:
        color_format = color_format.replace(elt, dict_color_rebrand[elt])
    return color_format

# Extractor function :
def extract_colors(colors):
    if not(pd.isnull(colors)):
        colors_l = colors.lower().replace(',', ' ').replace('&', ' ').replace('/', ' ').replace('-', ' ').translate(remove_digits).split()
        color_format = ''
        for color in colors_l:
            if color in webcolors.CSS3_NAMES_TO_HEX: 
                color_format += color +', '
        return color_rebrand(color_format)[:-2]
    else: return

# Some testing :
print(extract_colors(' Silver and Black&gold&siLver '))
print(extract_colors(' Silver/orange, dark grey23 and 10black '))
print(extract_colors(' ABD--Copper&&Stainless steel '))

silver, black, gold, silver
silver, orange, grey, black
copper, stainless steel


In [52]:
df['Color'] = df['Color'].apply(lambda colors : extract_colors(colors))
df['Color'].unique()

array(['black', 'stainless steel', 'grey', 'silver', '', 'green', None,
       'white, stainless steel', 'black, stainless steel', 'black, black',
       'white', 'black, silver', 'copper', 'teal', 'aqua',
       'stainless steel, black', 'orange', 'blue',
       'stainless steel, silver', 'silver, black', 'nickel', 'red', 'b',
       'black, gold', 'brown', 'yellow', 'gray'], dtype=object)

<a class="anchor" id="chapter5"></a>
## 5. Saving the final DataFrame

In [53]:
#Final look on the scrapped features / scrapped induced features :
df.columns

Index(['price', 'rating', 'timesRated', 'title', 'link', 'page', 'position',
       'delivery', 'Brand', 'Capacity', 'Color', 'Item Dimensions LxWxH',
       'Special Feature', 'Coffee Maker Type', 'Product Dimensions',
       'Material', 'Filter Type', 'Specific Uses For Product',
       'Recommended Uses For Product', 'Style', 'Number of settings',
       'Number of Programs', 'Included Components', 'Operation Mode',
       'Wattage', 'Voltage', 'Model Name', 'Number of Items', 'Unit Count',
       'Department', 'Human Interface Input', 'Package Type',
       'Package Dimensions', 'Item Weight', 'Power Source', 'Exterior Finish',
       'Manufacturer', 'ASIN', 'Domestic Shipping', 'Country of Origin',
       'Item model number', 'Batteries', 'Material Care Instructions',
       'Assembly Required', 'Number of Pieces', 'Batteries Required?',
       'Is Discontinued By Manufacturer', 'Date First Available',
       'Capacity (in liters)', 'contains_plastic', 'contains_metal',
       'co

In [54]:
# We first remove the columns that won't be useful for our future analysis :
columns_removed = []
df.drop(columns_removed, axis=1, inplace=True)

We then save our final DataFrame into a csv file.

In [56]:
# Saving the DataFrame :
df.to_csv('file1.csv')




















```
──────────────────────────────▓▓█───────
────────────────────────────▒██▒▒█──────
───────────────────────────█▓▓▓░▒▓▓─────
─────────────────────────▒█▓▒█░▒▒▒█─────
────────────────────────▒█▒▒▒█▒▒▒▒▓▒────
─▓▓▒░──────────────────▓█▒▒▒▓██▓▒░▒█────
─█▓▓██▓░──────────────▓█▒▒▒▒████▒▒▒█────
─▓█▓▒▒▓██▓░──────────▒█▒▒▒▒▒██▓█▓░░▓▒───
─▓▒▓▒▒▒▒▒▓█▓░──░▒▒▓▓██▒▒▒▒▒▒█████▒▒▒▓───
─▓░█▒▒▒▒▒▒▒▓▓█▓█▓▓▓▓▒▒▒▒▒▒▒▒██▓██▒░▒█───
─▓░▓█▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▓████▒▒▒█───
─▓░▓██▒▒▒▒▒▒▒▒▒▒▒▒▒▓▓▒▒▒▒▒▒▒▒▒▓██░░░█───
─▓░▓███▒▒▒▒▒▒▒▒▒▒▒▓█▒▒▒▒▒▒▒▒▒▒▒▒▓▓▓▒▓▓──
─▒▒▒██▓▒▓█▓▒▒▒▒▒▒▒▓▒▒▒▒▒▒▓▓▓▒▒▒▒▒▒▒▓▒█──
──▓▒█▓▒▒▒▒▓▒▒▒▒▒▒▒▒▒▒▒▓█▓▓▓▓█▓▒▒▒▒▒▒▒▓▒─
──▓▒█▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▓▓──────▓█▓▒▒▒▒▒▓█─
──▒▒▓▒▒▒▓▓▓▒▒▒▒▒▒▒▒▒▓▓───░▓▓───█▓▒▒▒▒▒█─
───█▒▒▓▓▓▒▒▓▓▒▒▒▒▒▒▓▓───█████▓──█▓▒▒▒▒▓▒
───▓▓█▒─────▒▓▒▒▒▒▒█───░██████──░█▒▒▒▒▓▓
───▓█▒──▒███─▒▓▒▒▒▒█────██████───▓▒▒▒▒▒▓
───██───█████─█▒▒▒▒█─────███▓────▓▓▒▒▒▒▓
───█▓───█████─▒▓▒▒▒█─────────────█▓▓▓▒▒▓
───█▓───░███──░▓▒▒▒▓█──────────░█▓▒▒▒▓▒▓
───██─────────▒▓▒▒▒▒▓▓──────░▒▓█▓────░▓▓
───▓█░────────█▓██▓▒▒▓█▓▓▓▓██▓▓▒▓▒░░▒▓▒▓
───▒██░──────▓▒███▓▒▒▒▒▓▓▓▓▒▒▒▒▒▒▓▓▓▓▒▓─
────█▓█▓▓▒▒▓█▓▒░██▒▒▓▓█▓▒▒▒▒▒▒▒▒▒▒▒▒▓▓█▒
────▓─░▓▓▓▓▓▒▓▓▓▓▒▓▓▓▒▓▒▒▒▒▒▒▒▒▒▒▒▓▓▓▓▓▓
────▒▒▒▓▒▒▒▒▒▒▓█░─░░░─▓▓▒▒▒▒▒▒▒▒▒▒▒▓██▓▒
─────█▓▒▒▒▒▒▒▒▒▓▓─░░░─▓▓▒▒▒▒▒▒▒▒▒▓▓▓▒▒▓▒
──────██▓▓▒▒▒▒▒▒█▒░░░░█▒▒▒▒▒▒▒▒▓█▓▓▒▒▒▒▒
─────░─▒██▓▓▒▒▒▒▒█▓▒▒▓▒▒▒▒▒▒▓███▓▒▒▒▒▒▓▓
──────────░▒▓▓▓▓▒▒▓▓▓▓▓▓████▓▓█▒▒▒▒▒▓▓█░
```
